In [59]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

## Whole Community Upgrade Cost Per Dwelling Unit

In [60]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    print(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [61]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [62]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    print(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [63]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    print(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    print(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    print(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    print(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [64]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [65]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [66]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [67]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [68]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    print(segment1_agg[['before', 'after']])
    print('segment2')
    print(segment2_agg[['before', 'after']])
    print('segment3')
    print(segment3_agg[['before', 'after']])
    print('segment4')
    print(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [69]:
community = 'louisville'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [70]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit
upgrade_name
Electric Cooking                                                    1000.8
Electric Clothes Dryer                                              1123.9
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Heat Pump Water Heater                                              2826.5
Basic Enclosure                                                     4978.4
Enhanced Enclosure                                                  6868.5
Mininum Efficiency Heat Pump with Existing Heat Backup             15391.5
Mininum Efficiency Heat Pump with Electric Heat Backup             15496.4
Mininum Efficiency Whole Home Electrification                      18101.8
High Efficiency Heat Pump with Electric Heat Backup                24498.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28878.8
Enhanced Enclosure + HPWH + High Efficiency HP/Elect

## 1.2 Total Annual Savings

In [71]:
figure_total_savings_1(community_results)

In [72]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [73]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [74]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight
upgrade_name
Electric Clothes Dryer                                              14795.580898
Electric Cooking                                                    86222.523165
Induction Cooking                                                  352542.979330
Heat Pump Clothes Dryer                                            307646.044191
Heat Pump Water Heater                                             357134.711333
Mininum Efficiency Whole Home Electrification                      253820.741269
Mininum Efficiency Heat Pump with Electric Heat Backup             356879.615110
Mininum Efficiency Heat Pump with Existing Heat Backup             356114.326443
Basic Enclosure                                                    349481.824661
Enhanced Enclosure                                                 352542.979330
High Efficiency Heat Pump with Electric Heat Backup                356879.615110
High Efficiency Whole Home Electrification                         357900.000000
B

# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single-Family Attached, 1940-1979, all income 

Segment 2. Occupied, Multifamily with 2-4 units, 1940-1979, all income 

Segment 3. Occupied, Single-Family Detached, 1940-1979, all income 

Segment 4. Occupied, Multifamily with 5+ units, 1940-1979, all income 


In [75]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Attached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 2 - 4 Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

## 2.1 Upgrade Cost Per Dwelling Unit 

In [76]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit
upgrade_name
Electric Cooking                                                    1000.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Basic Enclosure                                                     2502.8
Heat Pump Water Heater                                              2579.0
Enhanced Enclosure                                                  3158.3
Mininum Efficiency Whole Home Electrification                      16903.9
Mininum Efficiency Heat Pump with Existing Heat Backup             18377.4
Mininum Efficiency Heat Pump with Electric Heat Backup             18409.2
High Efficiency Heat Pump with Electric Heat Backup                20955.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        22282.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22499.5
Basic Enclosure + High Efficiency Whole 

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [77]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1
upgrade_name
Basic Enclosure                                                    0.10484
Enhanced Enclosure                                                 0.09267
Heat Pump Water Heater                                             0.03433
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03089
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03065
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02769
Basic Enclosure + High Efficiency Whole Home Electrification       0.02746
High Efficiency Whole Home Electrification                         0.02223
High Efficiency Heat Pump with Electric Heat Backup                0.02116
Heat Pump Clothes Dryer                                            0.01060
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00652
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00614
Mininum Efficiency Whole Home Electrification                      0.00570
Ind

 ### Annual Energy Savings Per Dollar Invested

In [78]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1
upgrade_name
Basic Enclosure                                                    0.00657
Enhanced Enclosure                                                 0.00595
Heat Pump Water Heater                                             0.00213
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00210
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00209
Mininum Efficiency Whole Home Electrification                      0.00199
Basic Enclosure + High Efficiency Whole Home Electrification       0.00185
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00185
High Efficiency Heat Pump with Electric Heat Backup                0.00169
High Efficiency Whole Home Electrification                         0.00164
Electric Cooking                                                   0.00151
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00140
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00112
Hea

 ### Annual Emission Savings Per Dollar Invested

In [79]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1
upgrade_name
Basic Enclosure                                                    0.50092
Enhanced Enclosure                                                 0.44748
Heat Pump Water Heater                                             0.15939
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.15201
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.15167
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13459
Basic Enclosure + High Efficiency Whole Home Electrification       0.13415
High Efficiency Heat Pump with Electric Heat Backup                0.11537
High Efficiency Whole Home Electrification                         0.11440
Mininum Efficiency Whole Home Electrification                      0.09209
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07177
Electric Cooking                                                   0.06273
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06103
Hea

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [80]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3851596.5
Basic Enclosure + High Efficiency Whole Home Electrification       3788044.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3722646.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3659094.2
High Efficiency Whole Home Electrification                         3216333.0
High Efficiency Heat Pump with Electric Heat Backup                2375256.1
Enhanced Enclosure                                                 1567979.5
Basic Enclosure                                                    1405711.2
Mininum Efficiency Heat Pump with Electric Heat Backup              642686.3
Mininum Efficiency Heat Pump with Existing Heat Backup              604102.0
Heat Pump Water Heater                                              474242.4
Mininum Efficiency Whole Home Electrification                       442688.8
Heat Pump Clothes Dryer                               

### Annual Total Energy Savings

In [81]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    256950.8
Basic Enclosure + High Efficiency Whole Home Electrification       255042.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     252465.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        250557.8
High Efficiency Whole Home Electrification                         237879.8
High Efficiency Heat Pump with Electric Heat Backup                189897.5
Mininum Efficiency Whole Home Electrification                      154613.3
Mininum Efficiency Heat Pump with Electric Heat Backup             137908.3
Mininum Efficiency Heat Pump with Existing Heat Backup             110735.2
Enhanced Enclosure                                                 100605.4
Basic Enclosure                                                     88100.8
Heat Pump Water Heater                                              29385.3
Heat Pump Clothes Dryer                                           

### Annual Total Emission Savings

In [82]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    18724287.4
Basic Enclosure + High Efficiency Whole Home Electrification       18507227.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     18321700.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        18104640.5
High Efficiency Whole Home Electrification                         16549152.2
High Efficiency Heat Pump with Electric Heat Backup                12950497.8
Enhanced Enclosure                                                  7571060.8
Mininum Efficiency Whole Home Electrification                       7148179.1
Mininum Efficiency Heat Pump with Electric Heat Backup              7077819.2
Basic Enclosure                                                     6716258.4
Mininum Efficiency Heat Pump with Existing Heat Backup              6008360.7
Heat Pump Water Heater                                              2202104.6
Heat Pump Clothes Dryer                   

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [83]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    719.0
Basic Enclosure + High Efficiency Whole Home Electrification       707.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     694.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        683.0
High Efficiency Whole Home Electrification                         600.4
High Efficiency Heat Pump with Electric Heat Backup                443.4
Enhanced Enclosure                                                 292.7
Basic Enclosure                                                    262.4
Mininum Efficiency Heat Pump with Electric Heat Backup             120.0
Mininum Efficiency Heat Pump with Existing Heat Backup             112.8
Mininum Efficiency Whole Home Electrification                       96.4
Heat Pump Water Heater                                              88.5
Heat Pump Clothes Dryer                                             24.0
Induction Cooking            

### Annual Energy Savings Per Dwelling Unit

In [84]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    48.0
Basic Enclosure + High Efficiency Whole Home Electrification       47.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     47.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        46.8
High Efficiency Whole Home Electrification                         44.4
High Efficiency Heat Pump with Electric Heat Backup                35.4
Mininum Efficiency Whole Home Electrification                      33.7
Mininum Efficiency Heat Pump with Electric Heat Backup             25.7
Mininum Efficiency Heat Pump with Existing Heat Backup             20.7
Enhanced Enclosure                                                 18.8
Basic Enclosure                                                    16.4
Heat Pump Water Heater                                              5.5
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                   

### Annual Percentage Energy Savings Per Dwelling Unit

In [85]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.3
Basic Enclosure + High Efficiency Whole Home Electrification       64.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     63.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        62.8
High Efficiency Whole Home Electrification                         59.6
High Efficiency Heat Pump with Electric Heat Backup                47.5
Mininum Efficiency Whole Home Electrification                      44.2
Mininum Efficiency Heat Pump with Electric Heat Backup             34.1
Mininum Efficiency Heat Pump with Existing Heat Backup             27.1
Enhanced Enclosure                                                 24.3
Basic Enclosure                                                    21.8
Heat Pump Water Heater                                              7.4
Electric Cooking                                                    1.8
Heat Pump Clothes Dryer                   

### Annual Emission Savings Per Dwelling Unit

In [86]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3495.3
Basic Enclosure + High Efficiency Whole Home Electrification       3454.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3420.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3379.6
High Efficiency Whole Home Electrification                         3089.2
High Efficiency Heat Pump with Electric Heat Backup                2417.5
Mininum Efficiency Whole Home Electrification                      1556.8
Enhanced Enclosure                                                 1413.3
Mininum Efficiency Heat Pump with Electric Heat Backup             1321.2
Basic Enclosure                                                    1253.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1121.6
Heat Pump Water Heater                                              411.1
Heat Pump Clothes Dryer                                              71.8
Electric Cooking

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [87]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [88]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [89]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [90]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [91]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1
                                                    before  after
upgrade_name                                                     
Enhanced Enclosure + High Efficiency Whole Home...     7.6    4.1
Basic Enclosure + High Efficiency Whole Home El...     7.6    4.2
Enhanced Enclosure + HPWH + High Efficiency HP/...     7.6    4.2
Basic Enclosure + HPWH + High Efficiency HP/Ele...     7.6    4.3
High Efficiency Whole Home Electrification             7.6    4.7
High Efficiency Heat Pump with Electric Heat Ba...     7.6    5.6
Enhanced Enclosure                                     7.6    6.0
Basic Enclosure                                        7.6    6.3
Mininum Efficiency Whole Home Electrification          7.8    7.2
Heat Pump Water Heater                                 7.6    7.2
Mininum Efficiency Heat Pump with Electric Heat...     7.6    7.2
Mininum Efficiency Heat Pump with Existing Heat...     7.6    7.3
Heat Pump Clothes Dryer 

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\4168045805.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\4168045805.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\4168045805.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\i

# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single-Family Attached, 1940-1979, 0-80% AMI  

Segment 2. Occupied, Multifamily with 2-4 units, 1940-1979, 0-80% AMI  

Segment 3. Occupied, Single-Family Detached, 1940-1979, 0-80% AMI  

Segment 4. Occupied, Multifamily with 5+ units, 1940-1979, 0-80% AMI   

In [92]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = 'specified_by_community'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Attached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Multi-Family with 2 - 4 Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &  
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

## 3.1 Upgrade Cost Per Dwelling Unit 

In [93]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit
upgrade_name
Electric Cooking                                                    1000.8
Induction Cooking                                                   1720.9
Heat Pump Clothes Dryer                                             2261.5
Basic Enclosure                                                     2431.1
Heat Pump Water Heater                                              2579.0
Enhanced Enclosure                                                  3291.5
Mininum Efficiency Heat Pump with Existing Heat Backup             17072.6
Mininum Efficiency Heat Pump with Electric Heat Backup             17114.3
Mininum Efficiency Whole Home Electrification                      17389.2
High Efficiency Heat Pump with Electric Heat Backup                20433.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        21967.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22252.2
Basic Enclosure + High Efficiency Whole 

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [94]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1
upgrade_name
Basic Enclosure                                                    0.10088
Enhanced Enclosure                                                 0.08659
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02978
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02945
Heat Pump Water Heater                                             0.02903
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02674
Basic Enclosure + High Efficiency Whole Home Electrification       0.02643
High Efficiency Whole Home Electrification                         0.02137
High Efficiency Heat Pump with Electric Heat Backup                0.01978
Heat Pump Clothes Dryer                                            0.01141
Mininum Efficiency Whole Home Electrification                      0.00564
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00525
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00499
Ind

 ### Annual Energy Savings Per Dollar Invested

In [95]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1
upgrade_name
Basic Enclosure                                                    0.00671
Enhanced Enclosure                                                 0.00588
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00212
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00212
Mininum Efficiency Whole Home Electrification                      0.00210
Heat Pump Water Heater                                             0.00209
Basic Enclosure + High Efficiency Whole Home Electrification       0.00187
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00187
Electric Cooking                                                   0.00174
High Efficiency Heat Pump with Electric Heat Backup                0.00170
High Efficiency Whole Home Electrification                         0.00167
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00148
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00111
Hea

 ### Annual Emission Savings Per Dollar Invested

In [96]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1
upgrade_name
Basic Enclosure                                                    0.49808
Enhanced Enclosure                                                 0.43152
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.15024
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14976
Heat Pump Water Heater                                             0.14719
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13314
Basic Enclosure + High Efficiency Whole Home Electrification       0.13254
High Efficiency Whole Home Electrification                         0.11354
High Efficiency Heat Pump with Electric Heat Backup                0.11250
Mininum Efficiency Whole Home Electrification                      0.09661
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07213
Electric Cooking                                                   0.07160
Mininum Efficiency Heat Pump with Existing Heat Backup             0.05770
Hea

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [97]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2801542.7
Basic Enclosure + High Efficiency Whole Home Electrification       2737990.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2704306.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2640755.0
High Efficiency Whole Home Electrification                         2305542.6
High Efficiency Heat Pump with Electric Heat Backup                1649513.7
Enhanced Enclosure                                                 1163306.8
Basic Enclosure                                                    1001038.4
Mininum Efficiency Heat Pump with Electric Heat Backup              366513.9
Mininum Efficiency Heat Pump with Existing Heat Backup              348061.5
Mininum Efficiency Whole Home Electrification                       325213.0
Heat Pump Water Heater                                              305608.6
Heat Pump Clothes Dryer                               

### Annual Total Energy Savings

In [98]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    195525.9
Basic Enclosure + High Efficiency Whole Home Electrification       193618.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     192226.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        190318.4
High Efficiency Whole Home Electrification                         180635.7
High Efficiency Heat Pump with Electric Heat Backup                141717.7
Mininum Efficiency Whole Home Electrification                      121185.0
Mininum Efficiency Heat Pump with Electric Heat Backup             103211.4
Enhanced Enclosure                                                  79058.1
Mininum Efficiency Heat Pump with Existing Heat Backup              77001.3
Basic Enclosure                                                     66553.6
Heat Pump Water Heater                                              21988.3
Heat Pump Clothes Dryer                                           

### Annual Total Emission Savings

In [99]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    13949070.4
Basic Enclosure + High Efficiency Whole Home Electrification       13732010.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     13644854.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        13427794.6
High Efficiency Whole Home Electrification                         12248352.4
High Efficiency Heat Pump with Electric Heat Backup                 9382693.5
Enhanced Enclosure                                                  5797125.9
Mininum Efficiency Whole Home Electrification                       5570971.7
Mininum Efficiency Heat Pump with Electric Heat Backup              5038651.0
Basic Enclosure                                                     4942323.5
Mininum Efficiency Heat Pump with Existing Heat Backup              4020897.0
Heat Pump Water Heater                                              1549342.6
Heat Pump Clothes Dryer                   

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3926536263.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [100]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    686.4
Basic Enclosure + High Efficiency Whole Home Electrification       670.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     662.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        647.0
High Efficiency Whole Home Electrification                         564.9
High Efficiency Heat Pump with Electric Heat Backup                404.1
Enhanced Enclosure                                                 285.0
Basic Enclosure                                                    245.3
Mininum Efficiency Whole Home Electrification                       98.1
Mininum Efficiency Heat Pump with Electric Heat Backup              89.8
Mininum Efficiency Heat Pump with Existing Heat Backup              85.3
Heat Pump Water Heater                                              74.9
Heat Pump Clothes Dryer                                             25.8
Induction Cooking            

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3071005957.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3071005957.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3071005957.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Annual Energy Savings Per Dwelling Unit

In [101]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    47.9
Basic Enclosure + High Efficiency Whole Home Electrification       47.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     47.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        46.6
High Efficiency Whole Home Electrification                         44.3
Mininum Efficiency Whole Home Electrification                      36.5
High Efficiency Heat Pump with Electric Heat Backup                34.7
Mininum Efficiency Heat Pump with Electric Heat Backup             25.3
Enhanced Enclosure                                                 19.4
Mininum Efficiency Heat Pump with Existing Heat Backup             18.9
Basic Enclosure                                                    16.3
Heat Pump Water Heater                                              5.4
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                   

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\396568384.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\396568384.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\396568384.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

### Annual Percentage Energy Savings Per Dwelling Unit

In [102]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.4
Basic Enclosure + High Efficiency Whole Home Electrification       63.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     63.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        62.8
High Efficiency Whole Home Electrification                         59.5
Mininum Efficiency Whole Home Electrification                      46.7
High Efficiency Heat Pump with Electric Heat Backup                46.4
Mininum Efficiency Heat Pump with Electric Heat Backup             33.0
Enhanced Enclosure                                                 25.1
Mininum Efficiency Heat Pump with Existing Heat Backup             24.2
Basic Enclosure                                                    21.8
Heat Pump Water Heater                                              7.6
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                   

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\1358844843.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\1358844843.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\1358844843.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Annual Emission Savings Per Dwelling Unit

In [103]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3417.6
Basic Enclosure + High Efficiency Whole Home Electrification       3364.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3343.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3289.9
High Efficiency Whole Home Electrification                         3000.9
High Efficiency Heat Pump with Electric Heat Backup                2298.8
Mininum Efficiency Whole Home Electrification                      1679.9
Enhanced Enclosure                                                 1420.3
Mininum Efficiency Heat Pump with Electric Heat Backup             1234.5
Basic Enclosure                                                    1210.9
Mininum Efficiency Heat Pump with Existing Heat Backup              985.1
Heat Pump Water Heater                                              379.6
Heat Pump Clothes Dryer                                              77.6
Electric Cooking

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\624873452.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\624873452.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\624873452.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [104]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

### Segment 2

In [105]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Segment 3

In [106]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Segment 4

In [107]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [108]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1
                                                    before  after
upgrade_name                                                     
Enhanced Enclosure + High Efficiency Whole Home...     9.2    5.0
Basic Enclosure + High Efficiency Whole Home El...     9.2    5.1
Enhanced Enclosure + HPWH + High Efficiency HP/...     9.2    5.1
Basic Enclosure + HPWH + High Efficiency HP/Ele...     9.2    5.2
High Efficiency Whole Home Electrification             9.2    5.8
High Efficiency Heat Pump with Electric Heat Ba...     9.2    6.8
Enhanced Enclosure                                     9.2    7.3
Basic Enclosure                                        9.2    7.7
Mininum Efficiency Whole Home Electrification         10.0    9.2
Heat Pump Water Heater                                 9.2    8.8
Mininum Efficiency Heat Pump with Electric Heat...     9.2    8.8
Mininum Efficiency Heat Pump with Existing Heat...     9.2    8.9
Heat Pump Clothes Dryer 

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\4168045805.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\4168045805.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17696\4168045805.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\i

In [109]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ylou2\\Desktop\\resstock\\euss_cleap\\results\\louisville\\saving_results_louisville.csv'